In [4]:
import pandas as pd


# po_start_range
# po_end_range
# Sample data
# data = {
#     'start_range': [0, 1500, 3000],
#     'end_range': [500, 4000, 4000],
#     'other_column': ['A', 'B', 'C']
# }

import pandas as pd
import xml.etree.ElementTree as ET
import psycopg2
from datetime import datetime
import re
import cx_Oracle
import numpy as np
# Database connection parameters
db_params = {
    "host": "gc-ue4-psql-cspo-dev01.nonprod.gcp.cswg.com",
    "database": "CSPODB",
    "port": 5432,
    "user": "cspoms",
    "password": "cspoms",
    "sslmode": "verify-ca",
    "sslcert": "D:\\W\\RELEX\\RELEX\\CSPOMS_DEV_Conn\\client-cert.pem",
    "sslkey": "D:\\W\\RELEX\\RELEX\\CSPOMS_DEV_Conn\\client-key.pem",
    "sslrootcert": "D:\\W\\RELEX\\RELEX\\CSPOMS_DEV_Conn\\server-ca.pem"
}

post_conn = psycopg2.connect(**db_params)
post_cur = post_conn.cursor()
source_query = '''SELECT * FROM XXPO_RANGE_CONFIG_MASTER'''

post_cur.execute(source_query)
data = post_cur.fetchall()
# Get column names from the cursor description
column_names = [desc[0] for desc in post_cur.description]

# Create DataFrame with data and column names
df = pd.DataFrame(data, columns=column_names)

print(df)
# Function to split ranges into chunks of 1000
def split_ranges(row):
    start = row['po_start_range']
    end = row['po_end_range']
    diff = int(end) - int(start)

    if diff <= 1000:
        return pd.DataFrame([row])
    
    new_rows = []
    current_start = start

    while current_start < end:
        current_end = min(current_start + 1000, end)
        new_row = row.copy()
        new_row['start_range'] = current_start
        new_row['end_range'] = current_end
        new_rows.append(new_row)
        current_start += 1000

    return pd.DataFrame(new_rows)

# Apply the function to each row
new_df = pd.concat([split_ranges(row) for _, row in df.iterrows()], ignore_index=True)

print(new_df)


    seq_no                  facility system  po_length po_start_range  \
0       30                       032     GS          6         012222   
1       32  014 - C&S HATFIELD NORTH     GS          6         010000   
2       33                       401     MF          6         020000   
3       45                      None     MF          6         210000   
4       59                      None     MF          6         001000   
5       62                      None   None          6         011000   
6       65                      None   None          6         011900   
7       67                      None   None          6          22222   
8       70                      None   None          6         002000   
9       71                      None   None          6         001010   
10      74                      None   None          6         938000   
11      81                      None   None          6         908001   
12      87                      None   None        

TypeError: can only concatenate str (not "int") to str

In [7]:
import pandas as pd
import psycopg2

# Database connection parameters
db_params = {
    "host": "gc-ue4-psql-cspo-dev01.nonprod.gcp.cswg.com",
    "database": "CSPODB",
    "port": 5432,
    "user": "cspoms",
    "password": "cspoms",
    "sslmode": "verify-ca",
    "sslcert": "D:\\W\\RELEX\\RELEX\\CSPOMS_DEV_Conn\\client-cert.pem",
    "sslkey": "D:\\W\\RELEX\\RELEX\\CSPOMS_DEV_Conn\\client-key.pem",
    "sslrootcert": "D:\\W\\RELEX\\RELEX\\CSPOMS_DEV_Conn\\server-ca.pem"
}

# Connect to the PostgreSQL database
post_conn = psycopg2.connect(**db_params)
post_cur = post_conn.cursor()

# Execute the query
source_query = '''SELECT * FROM XXPO_RANGE_CONFIG_MASTER'''
post_cur.execute(source_query)
data = post_cur.fetchall()

# Get column names from the cursor description
column_names = [desc[0] for desc in post_cur.description]

# Create DataFrame with data and column names
df = pd.DataFrame(data, columns=column_names)

# Function to split ranges into chunks of 1000
def split_ranges(row):
    start = int(row['po_start_range'])
    end = int(row['po_end_range'])
    diff = end - start

    if diff <= 1000:
        return pd.DataFrame([row])
    
    new_rows = []
    current_start = start

    while current_start < end:
        current_end = min(current_start + 1000, end)
        new_row = row.copy()
        new_row['po_start_range'] = current_start
        new_row['po_end_range'] = current_end -1
        new_rows.append(new_row)
        current_start += 1000

    return pd.DataFrame(new_rows)

# Apply the function to each row
new_df = pd.concat([split_ranges(row) for _, row in df.iterrows()], ignore_index=True)

print(new_df)

# Close the cursor and connection
post_cur.close()
post_conn.close()
# 1170

      seq_no facility system  po_length po_start_range po_end_range last_po  \
0         30      032     GS          6          12222        13221    None   
1         30      032     GS          6          13222        14221    None   
2         30      032     GS          6          14222        15221    None   
3         30      032     GS          6          15222        16221    None   
4         30      032     GS          6          16222        17221    None   
...      ...      ...    ...        ...            ...          ...     ...   
1166      90      252     GS          8       20091001     20092000    None   
1167      90      252     GS          8       20092001     20093000    None   
1168      90      252     GS          8       20093001     20094000    None   
1169      90      252     GS          8       20094001     20094998    None   
1170      24      022     GS          6         010011       010021    None   

                range_type vendor  ap_vendor_num ov